In [2]:
import pandas as pd

# Data inladen
df = pd.read_csv("groot_bestand.csv")

# Simpele value_score berekenen
df["value_score"] = df["points"] / (df["price"] + 1)  # of gebruik log prijs

def recommend_wine(variety="Chardonnay", max_price=None, top_n=3):
    # Filter op druif
    wines = df[df["variety"].str.contains(variety, case=False, na=False)].copy()
    
    # Filter op max prijs als opgegeven
    if max_price:
        wines = wines[wines["price"] <= max_price]
    
    # Sorteer op prijs/kwaliteit
    wines = wines.sort_values(by="value_score", ascending=False)
    
    return wines[["title", "variety", "country", "points", "price", "value_score"]].head(top_n)

# Voorbeeld
print(recommend_wine("Chardonnay", max_price=30, top_n=3))


                                                   title     variety country  \
31530                  Bandit NV Chardonnay (California)  Chardonnay      US   
59507    Pam's Cuties NV Unoaked Chardonnay (California)  Chardonnay      US   
8428   Earth's Harvest 2014 Organic Grapes Chardonnay...  Chardonnay      US   

       points  price  value_score  
31530      84    4.0    16.800000  
59507      83    4.0    16.600000  
8428       85    5.0    14.166667  


In [3]:
!pip install gradio


  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached Brotli-1.1.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached orjson-3.11.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (41 kB)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.48.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux201

In [4]:
import pandas as pd
import numpy as np
import gradio as gr

# 1. Data inladen
df = pd.read_csv("groot_bestand.csv")
df = df.dropna(subset=["points", "price", "country", "variety"])
df = df[(df["points"] > 0) & (df["price"] > 0)]
df["value_score"] = df["points"] / np.log1p(df["price"])

# 2. Dropdown keuzes
top_varieties = ["(Alle)"] + df["variety"].value_counts().head(20).index.tolist()
top_countries = ["(Alle)"] + df["country"].value_counts().head(20).index.tolist()

# 3. Wijn type mapping
def wine_type(variety):
    variety = variety.lower()
    if any(x in variety for x in ["pinot noir", "cabernet", "merlot", "syrah", "malbec", "tempranillo"]):
        return "Rood"
    if any(x in variety for x in ["chardonnay", "sauvignon blanc", "pinot grigio", "gruner", "albariño"]):
        return "Wit droog"
    if any(x in variety for x in ["riesling", "moscato", "gewürztraminer", "late harvest"]):
        return "Wit zoet"
    if any(x in variety for x in ["rosé", "rose"]):
        return "Rosé"
    if any(x in variety for x in ["champagne", "prosecco", "sparkling"]):
        return "Champagne"
    return "Overig"

df["wine_type"] = df["variety"].apply(wine_type)
wine_types = ["(Alle)", "Rood", "Wit droog", "Wit zoet", "Rosé", "Champagne"]

# 4. Label functie voor value_score
def label_value_score(score):
    if score >= 30:
        return "🌟 Top deal"
    elif score >= 20:
        return "✅ Goed"
    elif score >= 10:
        return "😐 Redelijk"
    else:
        return "💸 Duur voor wat je krijgt"

# 5. Aanbevelingsfunctie
def recommend(variety, country, wine_type_choice, min_price, max_price, top_n, sort_mode):
    wines = df.copy()
    if variety != "(Alle)":
        wines = wines[wines["variety"] == variety]
    if country != "(Alle)":
        wines = wines[wines["country"] == country]
    if wine_type_choice != "(Alle)":
        wines = wines[wines["wine_type"] == wine_type_choice]
    wines = wines[(wines["price"] >= min_price) & (wines["price"] <= max_price)]
    
    if wines.empty:
        return pd.DataFrame([{"Resultaat": "Geen wijnen gevonden"}])
    
    # Sorteren afhankelijk van de keuze
    if sort_mode == "Prijs/kwaliteit":
        wines = wines.sort_values(by=["value_score", "points"], ascending=[False, False])
    elif sort_mode == "Kwaliteit":
        wines = wines.sort_values(by="points", ascending=False)
    elif sort_mode == "Goedkoopste wijn":
        wines = wines.sort_values(by="price", ascending=True)
    
    # Resultaat samenstellen
    result = wines[["title", "variety", "wine_type", "country", "points", "price", "value_score"]].head(top_n)
    result["value_score"] = result["value_score"].round(1)
    result["value_label"] = result["value_score"].apply(label_value_score)
    return result

# 6. Gradio app
demo = gr.Interface(
    fn=recommend,
    inputs=[
        gr.Dropdown(choices=top_varieties, label="Kies een wijn"),
        gr.Dropdown(choices=top_countries, label="Kies land"),
        gr.Dropdown(choices=wine_types, label="Kies type"),
        gr.Slider(minimum=5, maximum=200, value=5, step=5, label="Min prijs ($)"),
        gr.Slider(minimum=5, maximum=200, value=30, step=5, label="Max prijs ($)"),
        gr.Number(value=3, label="Aantal aanbevelingen"),
        gr.Radio(["Prijs/kwaliteit", "Kwaliteit", "Goedkoopste wijn"], value="Prijs/kwaliteit", label="Zoek op")
    ],
    outputs=gr.Dataframe(label="Top aanbevelingen (beste opties bovenaan)")
)

demo.launch()


/workspaces/data-mining-2025/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
